# Preparing the Data for Analysis

This guide details how to access and prepare the different VanZyl-1 data products for further analysis. VZ-1 hosts two sensors: a visible and near-infrared imager (called VIRI) with a ground sample distance (GSD) of ~30 m, and a longwave infrared imager (called LIRI) with GSD ~70 m.

We'll start by accessing the STAC catalog so that we can query data for a specific area of interest (AOI) and time period of interest (TOI). See the ```vanzyl-1-quickstart``` or ```using-the-api``` guides for more detailed explanations of these steps.

In [1]:
# Import dependencies

import json
import requests
import pystac
from pystac_client import Client
import os
import base64
import matplotlib.pyplot as plt

In [2]:
# Retrieve credentials

with open('creds.json') as f:
    creds = json.loads(f.read())

In [3]:
# Connect to the catalog with your credentials

userpass = f"{creds['username']}:{creds['password']}"
b64 = base64.b64encode(userpass.encode()).decode()
headers = {'Authorization':'Basic ' + b64}

cat_url = 'https://stac.hydrosat.com'
catalog = Client.open(cat_url, headers)

The catalog contains different collections corresponding to the different VZ-1 data products.

These collections represent:
- Level 1A: Geolocated radiance (VIRI and LIRI data provided separately, with pixel resolution equivalent to GSD)
- Level 1B: Geolocated radiance (VIRI and LIRI data aligned and resampled to 30 m)
- Level 2: Surface reflectance and land surface temperature (30 m)

## Level 1A Products

Let's start by looking for data in the ```vz-l1a``` collection that intersects a specific AOI and TOI.

In [5]:
# Search for data that intersects your AOI and TOI

geom = {'type': 'Point', 'coordinates': [-120.211,36.535]}

search = catalog.search(
    collections = ["pydms_sharpened_landsat"],
    intersects = geom,
    datetime = ["2021-08-17", "2021-10-30"],
    max_items = 10
)

items = search.get_all_items()

print("Found " + str(len(list(items))) + " items.")

Found 8 items.


Each item returned by the search represents an individual scene. The item provides information about where and when the imagery was captured, as well as contains links to the imagery files. At L1A, each item contains either LIRI or VIRI data. At L1B and above, data from both sensors is aligned and provided within a single item.

In [ ]:
items[0]

The L1A data is the least processed of the available imagery. It includes processing applied onboard the instrument, such as time delay integration and non-uniformity corrections, as well as co-registration of sensor bands and georegistration. The full swath data is provided to users in image space with no resampling. 

Each image asset contains digital number (DN) values. To convert these to radiance, we need gain and offset coefficients from the companion metadata file.

## Level 1B Products

Now we'll search in a different collection, ```vz-l1b```, to explore the L1B data.

In [ ]:
search = catalog.search(
    collections = ["pydms_sharpened_landsat"],
    intersects = geom,
    datetime = ["2021-08-17", "2021-10-30"],
    max_items = 10
)

items = search.get_all_items()

print("Found " + str(len(list(items))) + " items.")

Like L1A, the L1B data contains geolocated radiance data. However, L1B processing also includes alignment of the VIRI and LIRI data, generation of a cloud mask, and orthorectification.

## Level 2 Products

Finally, let's dig into the surface reflectance and land surface temperature data provided at L2. This data is stored in the ```vz-l2``` collection.

In [ ]:
search = catalog.search(
    collections = ["pydms_sharpened_landsat"],
    intersects = geom,
    datetime = ["2021-08-17", "2021-10-30"],
    max_items = 10
)

items = search.get_all_items()

print("Found " + str(len(list(items))) + " items.")

The L2 items contain four imagery assets:
1. Per-band surface reflectance (SR)
2. Per-band emissivity
3. Land surface temperature (LST)
4. LST uncertainty